## Py-metrics

In [2]:
import motmetrics as mm
import numpy as np

In [3]:
# Create an accumulator that will be updated during each frame
acc = mm.MOTAccumulator(auto_id=True)

# Call update once for per frame. For now, assume distances between
# frame objects / hypotheses are given.
acc.update(
    [1, 2],                     # Ground truth objects in this frame
    [1, 2, 3],                  # Detector hypotheses in this frame
    [
        [0.1, np.nan, 0.3],     # Distances from object 1 to hypotheses 1, 2, 3
        [0.5,  0.2,   0.3]      # Distances from object 2 to hypotheses 1, 2, 3
    ]
)

0

In [4]:
print(acc.events)

                Type  OId  HId    D
FrameId Event                      
0       0        RAW  NaN  NaN  NaN
        1        RAW  1.0  1.0  0.1
        2        RAW  1.0  3.0  0.3
        3        RAW  2.0  1.0  0.5
        4        RAW  2.0  2.0  0.2
        5        RAW  2.0  3.0  0.3
        6      MATCH  1.0  1.0  0.1
        7      MATCH  2.0  2.0  0.2
        8         FP  NaN  3.0  NaN


In [5]:
print(acc.mot_events) # a pandas DataFrame containing MOT only events

                Type  OId  HId    D
FrameId Event                      
0       6      MATCH  1.0  1.0  0.1
        7      MATCH  2.0  2.0  0.2
        8         FP  NaN  3.0  NaN


In [6]:
mh = mm.metrics.create()
summary = mh.compute(acc, metrics=['num_frames', 'mota', 'motp'], name='acc')
print(summary)


     num_frames  mota  motp
acc           1   0.5  0.15


In [7]:
strsummary = mm.io.render_summary(
    summary,
    formatters={'mota' : '{:.2%}'.format},
    namemap={'mota': 'MOTA', 'motp' : 'MOTP'}
)
print(strsummary)

"""
      num_frames   MOTA      MOTP
full           3 50.00%  0.340000
part           2 50.00%  0.166667
"""

     num_frames   MOTA  MOTP
acc           1 50.00%  0.15


'\n      num_frames   MOTA      MOTP\nfull           3 50.00%  0.340000\npart           2 50.00%  0.166667\n'

In [8]:
summary = mh.compute_many(
    [acc, acc.events.loc[0:1]],
    metrics=mm.metrics.motchallenge_metrics,
    names=['full', 'part'],
    generate_overall=True
    )

strsummary = mm.io.render_summary(
    summary,
    formatters=mh.formatters,
    namemap=mm.io.motchallenge_metric_names
)
print(strsummary)



         IDF1   IDP    IDR   Rcll  Prcn GT MT PT ML FP FN IDs  FM  MOTA  MOTP IDt IDa IDm
full    80.0% 66.7% 100.0% 100.0% 66.7%  2  2  0  0  1  0   0   0 50.0% 0.150   0   0   0
part    80.0% 66.7% 100.0% 100.0% 66.7%  2  2  0  0  1  0   0   0 50.0% 0.150   0   0   0
OVERALL 80.0% 66.7% 100.0% 100.0% 66.7%  4  4  0  0  2  0   0   0 50.0% 0.150   0   0   0


In [10]:
#<frame number>, <object id>, <bb_left>, <bb_top>, <bb_width>, <bb_height>, <confidence>, <x>, <y>, <z>

In [9]:
def motMetricsEnhancedCalculator(gtSource, tSource):
  # import required packages
  import motmetrics as mm
  import numpy as np
  
  # load ground truth
  gt = np.loadtxt(gtSource, delimiter=',')

  # load tracking output
  t = np.loadtxt(tSource, delimiter=',')

  # Create an accumulator that will be updated during each frame
  acc = mm.MOTAccumulator(auto_id=True)

  # Max frame number maybe different for gt and t files
  for frame in range(int(gt[:,0].max())):
    frame += 1 # detection and frame numbers begin at 1

    # select id, x, y, width, height for current frame
    # required format for distance calculation is X, Y, Width, Height \
    # We already have this format
    gt_dets = gt[gt[:,0]==frame,1:6] # select all detections in gt
    t_dets = t[t[:,0]==frame,1:6] # select all detections in t

    C = mm.distances.iou_matrix(gt_dets[:,1:], t_dets[:,1:], \
                                max_iou=0.5) # format: gt, t

    # Call update once for per frame.
    # format: gt object ids, t object ids, distance
    acc.update(gt_dets[:,0].astype('int').tolist(), \
              t_dets[:,0].astype('int').tolist(), C)

  mh = mm.metrics.create()

  summary = mh.compute(acc, metrics=['num_frames', 'idf1', 'idp', 'idr', \
                                     'recall', 'precision', 'num_objects', \
                                     'mostly_tracked', 'partially_tracked', \
                                     'mostly_lost', 'num_false_positives', \
                                     'num_misses', 'num_switches', \
                                     'num_fragmentations', 'mota', 'motp' \
                                    ], \
                      name='acc')

  strsummary = mm.io.render_summary(
      summary,
      #formatters={'mota' : '{:.2%}'.format},
      namemap={'idf1': 'IDF1', 'idp': 'IDP', 'idr': 'IDR', 'recall': 'Rcll', \
               'precision': 'Prcn', 'num_objects': 'GT', \
               'mostly_tracked' : 'MT', 'partially_tracked': 'PT', \
               'mostly_lost' : 'ML', 'num_false_positives': 'FP', \
               'num_misses': 'FN', 'num_switches' : 'IDsw', \
               'num_fragmentations' : 'FM', 'mota': 'MOTA', 'motp' : 'MOTP',  \
              }
  )
  print(strsummary)

In [10]:
# Calculate the MOT metrics
motMetricsEnhancedCalculator('motmetrics/data/TUD-Campus/gt.txt', 'motmetrics/data/TUD-Campus/test.txt')

     num_frames      IDF1      IDP       IDR      Rcll      Prcn   GT  MT  PT  ML  FP   FN  IDsw  FM      MOTA      MOTP
acc          71  0.557659  0.72973  0.451253  0.582173  0.941441  359   1   6   1  13  150     7   7  0.526462  0.277201
